In [ ]:
from seg_models.micronet_backbone import micronet
from seg_models.micronet_backbone import cfg
import seg_models as sm
import torch
import utils
import torchio as tio
import SimpleITK as sitk
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

In [ ]:
infect_path = "data/Infection Segmentation Data Transformed/"
infect_train_loader, infect_val_loader, infect_test_loader = utils.get_loader(
    target_directory=infect_path, train_bs=2,
    val_bs=2, num_works=2)
lung_path = "data/Lung Segmentation Data Transformed/"
lung_train_loader, lung_val_loader, lung_test_loader = utils.get_loader(
    target_directory=lung_path, train_bs=64,
    val_bs=64, num_works=2)
len(infect_train_loader.dataset), len(lung_train_loader.dataset)

In [ ]:
model = sm.Unet(
    encoder_name="micronet_m0",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                      # model output channels (number of classes in your dataset)
)
optimizer = optim.Adam(lr=0.001)
loss_fnc = nn.CrossEntropyLoss()

In [ ]:
infect_value = next(iter(infect_train_loader))
lung_value = next(iter(lung_train_loader))

In [ ]:
infect_value['infection mask'][tio.DATA].shape, lung_value['lung mask'][tio.DATA].shape

In [ ]:
# check loss function calculation
imgs = lung_value['image'][tio.DATA].squeeze_(-1)
labels = lung_value['lung mask'][tio.DATA].squeeze_(1)
labels = labels.squeeze_(-1).long()
outputs = model(imgs)
_, predicted = torch.max(outputs, dim=1)
print(outputs.shape, labels.shape)
loss = nn.CrossEntropyLoss()
print(loss(outputs, labels).item())

## Test training_loop

In [ ]:
Unet_micronet_m0 = sm.Unet(
    encoder_name="micronet_m0",       
    in_channels=1,                  
    classes=2,                      
)
optimizer = optim.Adam(Unet_micronet_m0.parameters(), lr=0.001)
loss_fcn = nn.CrossEntropyLoss()

In [ ]:
utils.training_loop(Unet_micronet_m0, optimizer, 
loss_fcn, infect_train_loader, 
infect_val_loader, mask_name='infection mask')

## Test test_loop

In [ ]:
utils.test_loop(infect_test_loader, model, mask_name='infection mask');

## test metric accuracy

In [ ]:
# 
imgs = infect_value['image'][tio.DATA].squeeze_(-1)
labels = infect_value['infection mask'][tio.DATA].squeeze_(-1).long()
outputs = model(imgs)
_, predicted = torch.max(outputs, dim=1)
print(predicted.view(-1).shape, labels.view(-1).shape)
print(utils.accuracy(predicted, labels), utils.IoU(predicted, labels), utils.DSC(predicted, labels))

In [ ]:
import numpy as np
def calc_Accuracy_Sets(truth, pred, c=1, **kwargs):
    # Obtain sets with associated class
    gt = np.equal(truth, c)
    pd = np.equal(pred, c)
    not_gt = np.logical_not(gt)
    not_pd = np.logical_not(pd)
    # Calculate Accuracy
    acc = (np.logical_and(pd, gt).sum() + \
           np.logical_and(not_pd, not_gt).sum()) / gt.size
    # Return computed Accuracy
    return acc
def calc_IoU_Sets(truth, pred, c=1, **kwargs):
    # Obtain sets with associated class
    gt = np.equal(truth, c)
    pd = np.equal(pred, c)
    # Calculate IoU
    if  (pd.sum() + gt.sum() - np.logical_and(pd, gt).sum()) != 0:
        iou = np.logical_and(pd, gt).sum() / \
              (pd.sum() + gt.sum() - np.logical_and(pd, gt).sum())
    else : iou = 0.0
    # Return computed IoU
    return iou
def calc_DSC_Sets(truth, pred, c=1, **kwargs):
    # Obtain sets with associated class
    gt = np.equal(truth, c)
    pd = np.equal(pred, c)
    # Calculate Dice
    if (pd.sum() + gt.sum()) != 0:
        dice = 2*np.logical_and(pd, gt).sum() / (pd.sum() + gt.sum())
    else : dice = 0.0
    # Return computed Dice
    return dice

In [ ]:
np.random.seed(1)
# Get some ground truth / annotated segmentations
truth = np.random.randint(2, size=(64,64))  # binary (2 classes)
# Get some predicted segmentations
np.random.seed(2)
pred = np.random.randint(2, size=(64,64))  # binary (2 classes)

print(calc_Accuracy_Sets(truth, pred), utils.accuracy(torch.tensor(pred), torch.tensor(truth)))
print(calc_IoU_Sets(truth, pred), utils.IoU(torch.tensor(pred), torch.tensor(truth)))
print(calc_DSC_Sets(truth, pred), utils.DSC(torch.tensor(pred), torch.tensor(truth)))

In [ ]:
a = torch.tensor([[0,1],[1,0]])
b = torch.tensor([[0,1],[0,1]])
((a+b)>0).sum()